Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## TODO

* Wrap all Exchange Order responses in an Order object
* Market orders do not return price. So fetch trades. https://github.com/ccxt/ccxt/issues/798
* Verify starting_cash <= actual balance on Live Exchange
* Handle/prevent 'insufficient funds' from Binance/Poloniex (Above will help, but still need to catch if the exchange balance has changed for some reason)
* Handle/prevent 'minimum order not met' from Binanace/Poloniex (method to refresh and lookup the minimum amount)
* Method to audit/confirm our local exchange balance reflects actual changes to CCXT exchange balance
* Limit Sell Orders not working I think..


## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.ETH, c.BTC)
        price = data['close']
        quantity = .01

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, quantity, price)
            orders.append(order)
        elif (len(ctx.record.portfolio.positions) > 0 
              and ctx.record.balance.get(c.ETH)[BalanceType.FREE] > 0.0):
            order = order_manager.build_market_sell_order(
                ctx.exchange, asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []
        
        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [2]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = CSVDataFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
strategy = SimpleStrategy()
experiment_name = 'default_backtest'

In [3]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-09T00:00:00
---------------------------------------
OHLCV
    O: 0.0760 | C: 0.0766 | V: 7900.8 | T: 2018-01-09T00:00:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-09T00:30:00
---------------------------------------
OHLCV
    O: 0.0766 | C: 0.0774 | V: 7715.3 | T: 2018-01-09T00:30:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 3 - Timestep: 2018-01-09T01:00:00
---------------------------------------
OHLCV
    O: 0.0774 | C: 0.0769 | V: 10114.2 | T: 2018-01-09T01:00:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 Pn

Placing Order {
    "id": "81934f6c86db464f89ccea57a18476f1",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.077252,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:14:04.074641",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "81934f6c86db464f89ccea57a18476f1",
    "exchange_id": "paper",
    "exchange_order_id": "e9f986fcdae04a60a6b43221b7119874",
    "price": 0.077252,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:14:04.074641",
    "opened_time": null,
    "filled_time": "2018-01-18T07:14:04.119227",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Equal? True
Placing Order {
    "id": "

---------------------------------------
Epoch 9 - Timestep: 2018-01-09T04:00:00
---------------------------------------
OHLCV
    O: 0.0791 | C: 0.0790 | V: 9275.8 | T: 2018-01-09T04:00:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0000157799999998, 'used': 0.0, 'total': 1.0000157799999998}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.0, 'cost_price': 0.0, 'latest_price': 0.07911}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 10 - Timestep: 2018-01-09T04:30:00
---------------------------------------
OHLCV
    O: 0.0790 | C: 0.0791 | V: 5713.5 | T: 2018-01-09T04:30:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0000157799999998, 'used': 0.0, 'total': 1.0000157799999998}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quanti

Placing Order {
    "id": "907a70d016e34c12b536dfbcde22e537",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.079539,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:14:04.381479",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "907a70d016e34c12b536dfbcde22e537",
    "exchange_id": "paper",
    "exchange_order_id": "9131ac014a90454bacfc2c3b8fb0541b",
    "price": 0.079539,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:14:04.381479",
    "opened_time": null,
    "filled_time": "2018-01-18T07:14:04.421800",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Equal? True
Placing Order {
    "id": "

---------------------------------------
Epoch 16 - Timestep: 2018-01-09T07:30:00
---------------------------------------
OHLCV
    O: 0.0799 | C: 0.0805 | V: 6801.3 | T: 2018-01-09T07:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9976 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99523072000000012, 'used': 0.0, 'total': 0.99523072000000012}
    ETH - {'free': 0.060000000000000005, 'used': 0.0, 'total': 0.060000000000000005}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.03, 'cost_price': 0.07981450000000001, 'latest_price': 0.080004999999999993}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 17 - Timestep: 2018-01-09T08:00:00
---------------------------------------
OHLCV
    O: 0.0805 | C: 0.0812 | V: 9038.7 | T: 2018-01-09T08:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9984 Total Val: 1.0000 PnL: 0.0000 Retu

...

Placing Order {
    "id": "25fc873329494c6f972b7cf1e1c0925a",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T07:14:11.946066",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "25fc873329494c6f972b7cf1e1c0925a",
    "exchange_id": "paper",
    "exchange_order_id": "4cbc8315acbe4e16a4bac18f564f0e39",
    "price": 0.087594,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T07:14:11.946066",
    "opened_time": null,
    "filled_time": "2018-01-18T07:14:11.988398",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Equal? True
Placing Order {
    "id": "e

    ETH - {'free': 0.11999999999999998, 'used': 0.0, 'total': 0.11999999999999998}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.060000000000000005, 'cost_price': 0.087942744444444415, 'latest_price': 0.086863999999999997}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 144 - Timestep: 2018-01-11T23:30:00
---------------------------------------
OHLCV
    O: 0.0860 | C: 0.0860 | V: 5216.2 | T: 2018-01-11T23:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0860 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9957 Total Val: 1.0000 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.99143378000000082, 'used': 0.0, 'total': 0.99143378000000082}
    ETH - {'free': 0.09999999999999999, 'used': 0.0, 'total': 0.09999999999999999}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.087942744444444415, 'latest_price': 0.086046999999999998}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 145 - Timestep: 

Placing Order {
    "id": "14969e710d294402a13544443b67f9df",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T07:14:12.146197",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "14969e710d294402a13544443b67f9df",
    "exchange_id": "paper",
    "exchange_order_id": "6868300df3404ed393873cf1445ae4ea",
    "price": 0.086047,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T07:14:12.146197",
    "opened_time": null,
    "filled_time": "2018-01-18T07:14:12.193673",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Equal? True
Placing Order {
    "id": "5

OHLCV
    O: 0.0941 | C: 0.0944 | V: 4164.2 | T: 2018-01-16T02:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0944 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9958 Total Val: 1.0005 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.9915841800000007, 'used': 0.0, 'total': 0.9915841800000007}
    ETH - {'free': 0.09999999999999999, 'used': 0.0, 'total': 0.09999999999999999}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.087613953703703673, 'latest_price': 0.093490000000000004}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 147 - Timestep: 2018-01-16T02:30:00
---------------------------------------
OHLCV
    O: 0.0945 | C: 0.0949 | V: 2114.9 | T: 2018-01-16T02:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9948 Total Val: 1.0005 PnL: 0.0005 Returns: 0.0005
BALANCE
    BTC - {'free': 0.98969566000000064, 'used': 0.0, 'total': 0.98969566000000064}
    ETH - {'free': 0.1

Placing Order {
    "id": "260d0cbd0a9346c7b691b6cc81bbacb3",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.094426,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:14:12.359943",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "260d0cbd0a9346c7b691b6cc81bbacb3",
    "exchange_id": "paper",
    "exchange_order_id": "5d73069b79114599a0c61faf4d249cb6",
    "price": 0.094426,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:14:12.359943",
    "opened_time": null,
    "filled_time": "2018-01-18T07:14:12.400137",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Equal? True
Placing Order {
    "id": "

## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.ETH, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
def live(name, exchange, balance, portfolio, feed, strategy):
    print("WARNING LIVE TRADING!!!!")
    '''
    exp_name = name of your current experiment (multiple runs per strategy)
    '''
    # Where we will save the record
    root = os.path.join(cfg.DATA_DIR, name)

    # This can be retrieved from the user's global config
    store = DATA_STORES[cfg.DATA_STORE](root=root)

    config = {
        'experiment': name, 
        'strategy': strategy.name,
    }
    record = Record(
        config=config,
        portfolio=portfolio,
        balance=balance,
        store=store
    )
    ctx = Context(
        exchange=exchange,
        feed=feed,
        record=record
    )
    feed.initialize()
    
    while True:
        row = feed.next()
        
        if row is not None:
            orders = strategy.process(row, ctx)

            # TODO: Cancelling orders
            # should we auto-cancel any outstanding orders
            # or should we leave this decision up to the Strategy?
            order_manager.cancel_orders(exchange, orders['cancel_ids'])

            # Returns both FILLED and PENDING orders
            # TODO: Order manager handles mapping from Exchange JSON
            # Particularly order types like CLOSED --> FILLED,
            # And OPEN vs PENDING <-- check the 'quantity' vs 'filled' amounts
            orders = order_manager.place_orders(exchange, orders['orders'])
            filled_orders = order_manager.get_filled_orders(orders)

            # Getting the latest prices for each of our positions
            positions = record.portfolio.positions
            latest_prices = get_latest_prices(positions, row)

            # Portfolio needs to know about new filled orders
            record.portfolio.update(filled_orders, latest_prices)

            # Record needs to know about all new orders
            for order in orders:
                record.orders[order.id] = order

            # Update Virtual Balance
            # exchange balance may be impacted by external trading
            for order in filled_orders:
                record.balance.update_by_order(order)

            record.save()
        
        time.sleep(30)
            
    return record

In [ ]:
cash = .04
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_exchange(c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.XRP, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_live'

In [ ]:
live(experiment_name, exchange, balance, portfolio, feed, strategy)